In [ ]:
!pip install -q --upgrade google-generativeai langchain-google-genai

In [ ]:
!pip install chromadb

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install langchain-core

In [ ]:
from google.colab import userdata
API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=API_KEY)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

DATA_PATH = "/content/SampleFoodBusinessPlanOklahomaState.pdf"

def load_document():
  loader = PyPDFLoader(DATA_PATH)
  documents = loader.load()
  return documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,
      chunk_overlap=500,
      length_function = len,
      add_start_index = True
      )

chunks = text_splitter.split_documents(load_document())
document = chunks[0]

print(document.page_content)
print(document.metadata)



A SAMPLE BUSINESS PLAN FOR 
SMALL FOOD BUSINESSES 
 
 
 
Rodney B. Holcomb 
Associate Professor, Dept. of Agricultural Economics 
Browning Endowed Professor of Food Science, Food & Agricultural Products Center 
 
Philip Kenkel 
Professor, Dept. of Agricultural Economics 
Bill Fitzwater Endowed Chair for Cooperative Studies 
 
Linda Blan-Byford 
(Former) Business Planning and Marketing Associate 
Food & Agricultural Products Center 
 
Oklahoma State University 
 
January 2006
{'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2017-02-05T16:13:08-05:00', 'title': 'A SAMPLE BUSINESS PLAN FOR', 'author': 'Rodney Holcomb', 'moddate': '2017-02-05T16:13:08-05:00', 'source': '/content/SampleFoodBusinessPlanOklahomaState.pdf', 'total_pages': 17, 'page': 0, 'page_label': '1', 'start_index': 0}


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001',google_api_key=API_KEY)

In [ ]:
from langchain.vectorstores import Chroma
vector_store = Chroma.from_documents(documents=chunks, embedding=embeddings)

In [ ]:
from langchain.chains import RetrievalQA
rag = RetrievalQA.from_chain_type(
    model,
    retriever = vector_store.as_retriever(),
    return_source_documents=True
)

In [ ]:

results = vector_store.similarity_search_with_relevance_scores(
    question,k=3
    )
if len(results) == 0 or results[0][1] < 0.7:
  print(f"unable to find matching results.")
  # return

unable to find matching results.


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [ ]:
prompt_template = PromptTemplate(
    input_variables=['context','query'],
    template='''
    answer the question based only on the following context:{context}

    answer the question based on the above contex: {query}

    Answer:
    '''
)

rag_chain = (
    {"context": rag, "query": RunnablePassthrough()}
    | prompt_template
    | model
    | StrOutputParser()
)

question = input('enter your question')
rag_response = rag.invoke({"query":question})
final_answer = rag_chain.invoke(question)


print(f"Question from human: {question}")
print(f"Response from RAG: {rag_response['result']}")
print(f"Final answer from LLM: {final_answer}")

enter your questionwhat are plans?
Question from human: what are plans?
Response from RAG: Based on the context, there are a few types of plans discussed:

*   **Business Plan:** A detailed blueprint for the activities needed to establish a business. It outlines the product/service, the market, and the management of the business. It can also be used to measure success, obtain loans, or attract venture capital.
*   **Manufacturing Plans:** These plans outline how a product will be manufactured, including facilities and processes.
*   **Marketing Strategies:** These are plans that describe the strategies for marketing a business.
*   **Contingency Plans:** Plans that prepare a business for unexpected events.
Final answer from LLM: Based on the context, "plans" can refer to:

1.  **Business Plans:** A detailed blueprint for the activities needed to establish a business, outlining the product/service, the market, and the management of the business. They are used for measuring success and o